<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Machine Learning Para Aplicações Biomédicas</font>
## <font color='blue'>Projeto 4</font>
### <font color='blue'>Ajuste Fino de LLM Para Medical Natural Language Inference (NLI)</font>

## Pacotes Python Usados no Projeto

In [1]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# !pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark.
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 56.7 MB/s eta 0:00:00


In [2]:
!pip install -q accelerate peft bitsandbytes transformers trl datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 99.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
# Imports
import accelerate
import peft
import bitsandbytes
import transformers
import trl
import datasets

In [4]:
# Imports
import os
import torch
from time import perf_counter
from datasets import load_dataset, Dataset
from peft import LoraConfig, AutoPeftModelForCausalLM, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from transformers import TrainingArguments, GenerationConfig
from trl import SFTTrainer
import warnings
warnings.filterwarnings('ignore')

In [5]:
%reload_ext watermark
%watermark -a "Data Science Academy"

Author: Data Science Academy



## Carregando e Compreendendo os Dados Para o Ajuste Fino

https://huggingface.co/datasets/nlpie/Llama2-MedTuned-Instructions

In [6]:
# Nome do dataset
dataset = "nlpie/Llama2-MedTuned-Instructions"

In [7]:
# Definindo a função dsa_prepara_dados com dois argumentos:
# data_id e num_samples com um valor padrão de 1000
def dsa_prepara_dados(data_id, num_samples = 1000):

    # Carrega o conjunto de dados especificado pelo data_id com a divisão de treino
    data = load_dataset(data_id, split = "train")

    # Seleciona um subconjunto dos dados com o mínimo entre num_samples e o tamanho total dos dados
    data = data.select(range(min(num_samples, len(data))))

    # Converte os dados selecionados em um DataFrame do pandas
    data_df = data.to_pandas()

    # Adiciona uma nova coluna "text" combinando as colunas "instruction",
    # "input", "output" com um formato específico
    data_df["text"] = data_df[["instruction", "input", "output"]].apply(lambda x: "user\n" + x["instruction"] + "  \n input\n" + x["input"] + "\nassistant\n" + x["output"] + "\n", axis=1)

    # Converte o DataFrame modificado de volta para um conjunto de dados do Hugging Face
    data = Dataset.from_pandas(data_df)

    # Retorna o conjunto de dados modificado
    return data

In [8]:
# Aplica a função
dsa_dados = dsa_prepara_dados(dataset, num_samples = 1000)

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-a8790d88efc2bc(…):   0%|          | 0.00/91.1M [00:00<?, ?B/s]

data/validation-00000-of-00001-b543c64b1(…):   0%|          | 0.00/6.08M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/200252 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/70066 [00:00<?, ? examples/s]

In [9]:
# Conjunto de dados
dsa_dados

Dataset({
    features: ['instruction', 'input', 'output', 'source', 'text'],
    num_rows: 1000
})

In [10]:
# Um exemplo completo
dsa_dados[0]

{'instruction': "In your role as a medical professional, address the user's medical questions and concerns.",
 'input': 'My relative suffering from secondary lever cancer ( 4th stage as per Allopathic doctor) and primary is in rectum. He is continuously with 103 to 104 degree F fever. Allpathic doctor suggested chemo only after fever subsidises. Is treatment possible at Lavanya & what is the time scale of recover.',
 'output': 'Hi, dairy have gone through your question. I can understand your concern. He has rectal cancer with liver metastasis. It is stage 4 cancer. Surgery is not possible at this stage. Only treatment options are chemotherapy and radiotherapy according to type of cancer. Inspite of all treatment prognosis is poor. Life expectancy is not good. Consult your doctor and plan accordingly. Hope I have answered your question, if you have any doubts then contact me at bit.ly/ Chat Doctor. Thanks for using Chat Doctor. Wish you a very good health.',
 'source': 'chatdoctor',
 't

## Carregando Modelo e Tokenizador

https://huggingface.co/TinyLlama/TinyLlama-1.1B-Chat-v1.0

In [11]:
# Define o nome do LLM
nome_llm = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

In [12]:
# Define a função que recebe um identificador de modelo como argumento
def dsa_get_model_and_tokenizer(mode_id):

    # Carrega o tokenizador para o identificador de modelo fornecido
    tokenizador = AutoTokenizer.from_pretrained(mode_id)

    # Define o token de preenchimento do tokenizador como seu token de fim de sequência
    tokenizador.pad_token = tokenizador.eos_token

    # Configura a quantização para o modelo usando BitsAndBytes para melhor eficiência de memória e computação
    bnb_config = BitsAndBytesConfig(load_in_4bit = True,  # Habilita o carregamento de modelos em 4 bits
                                    bnb_4bit_quant_type = "nf4",  # Define o tipo de quantização para "nf4"
                                    bnb_4bit_compute_dtype = "float16",  # Define o tipo de dados de computação para float16
                                    bnb_4bit_use_double_quant = True)  # Habilita a quantização dupla para maior precisão

    # Carrega o modelo de linguagem causal com a configuração de quantização especificada
    # e mapeamento automático de dispositivo
    modelo = AutoModelForCausalLM.from_pretrained(mode_id,
                                                  quantization_config = bnb_config,
                                                  device_map = "auto")

    # Desabilita o uso do cache pelo modelo para reduzir o uso de memória
    modelo.config.use_cache = False

    # Define o parâmetro de treinamento de previsão para 1, ajustando o comportamento de previsão do modelo
    modelo.config.pretraining_tp = 1

    # Retorna o modelo e o tokenizador
    return modelo, tokenizador

In [13]:
# Aplica a função
dsa_model, dsa_tokenizer = dsa_get_model_and_tokenizer(nome_llm)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

## Parâmetros LoRa

In [14]:
# LoRA (Low-Rank Adaptation)
dsa_peft_config = LoraConfig(r = 8,
                             lora_alpha = 16,
                             lora_dropout = 0.05,
                             bias = "none",
                             task_type = "CAUSAL_LM")

A configuração dsa_peft_config usa a LoraConfig, que é um conjunto de parâmetros para configurar a LoRA (Low-Rank Adaptation), uma técnica de adaptação de modelos de linguagem para tarefas específicas. Vamos detalhar cada um dos parâmetros:
<!-- Projeto Desenvolvido na Data Science Academy - www.datascienceacademy.com.br -->
**r (rank)**: Especifica o rank da adaptação LoRA. Um valor menor de r implica uma matriz de adaptação de menor dimensão, o que reduz o número de parâmetros adicionais e a complexidade computacional. Aqui, r = 8 significa que a matriz de adaptação LoRA terá um rank de 8, buscando um equilíbrio entre eficiência e eficácia na adaptação.

**lora_alpha (scale factor)**: Este é o fator de escala para os pesos adaptados LoRA. Um lora_alpha maior aumenta o impacto dos parâmetros LoRA adaptados na transformação da entrada. lora_alpha = 16 indica uma amplificação significativa, permitindo que as adaptações LoRA tenham um impacto mais substancial nas ativações.

**lora_dropout (dropout rate)**: Taxa de dropout aplicada aos pesos adaptados LoRA. O dropout é uma técnica de regularização usada para prevenir o sobreajuste, descartando aleatoriamente uma proporção dos pesos durante o treinamento. Aqui, lora_dropout = 0.05 significa que 5% dos pesos adaptados serão zerados aleatoriamente, ajudando a melhorar a generalização do modelo.

**bias**: Define como o viés é tratado na adaptação LoRA. bias = "none" indica que nenhum viés adicional será usado na adaptação LoRA, mantendo o foco exclusivamente nos pesos adaptativos.

**task_type**: Especifica o tipo de tarefa para a qual o modelo está sendo adaptado. task_type = "CAUSAL_LM" indica que a adaptação está sendo feita para uma tarefa de modelagem de linguagem causal, onde o modelo gera texto baseado no contexto anterior, considerando apenas a informação disponível antes do token atual na geração de texto.

Esses parâmetros são usados para ajustar o modelo a uma tarefa específica de forma eficiente, minimizando a necessidade de treinamento extensivo em grandes conjuntos de dados e permitindo personalizações específicas para melhorar o desempenho em tarefas de modelagem de linguagem causal.

## Parâmetros de Treino

In [15]:
# Nome do modelo de saída
output_model = "tinyllama-dsa-v1"

In [16]:
# Argumentos de treino
dsa_training_arguments = TrainingArguments(output_dir = output_model,
                                           per_device_train_batch_size = 1,
                                           gradient_accumulation_steps = 4,
                                           optim = "paged_adamw_32bit",
                                           learning_rate = 2e-4,
                                           lr_scheduler_type = "cosine",
                                           save_strategy = "epoch",
                                           logging_steps = 10,
                                           num_train_epochs = 3,
                                           max_steps = 250,
                                           fp16 = True,
                                           report_to="none")

A variável dsa_training_arguments está configurando TrainingArguments para o treinamento do modelo com a biblioteca Transformers. Cada parâmetro tem um propósito específico na configuração do treinamento. Vamos entender o significado de cada um deles:

**output_dir**: Caminho para o diretório onde os modelos treinados e os checkpoints serão salvos. output_model indica que a saída será armazenada no diretório especificado pela variável output_model.

**per_device_train_batch_size**: Tamanho do lote (batch size) para o treinamento em cada dispositivo. Definir como 1 significa que cada dispositivo processará um exemplo de treinamento por vez. Isso é útil para lidar com limitações de memória, especialmente em modelos grandes.

**gradient_accumulation_steps**: Número de passos para acumular os gradientes antes de realizar uma etapa de otimização. Aqui, 4 passos são acumulados. Isso permite efetivamente aumentar o tamanho do lote (batch size) sem aumentar a memória necessária, combinando os gradientes de várias etapas de mini-lotes antes da atualização dos parâmetros.

**optim**: Especifica o otimizador a ser usado. paged_adamw_32bit indica o uso de uma versão otimizada do AdamW para treinamento mais eficiente, especialmente projetada para funcionar bem com precisão de 32 bits.

**learning_rate**: Taxa de aprendizado inicial. 2e-4 (ou 0.0002) é um valor comum para muitas tarefas de PLN, oferecendo um bom equilíbrio entre rapidez de convergência e risco de "pular" o mínimo global.

**lr_scheduler_type**: Tipo de agendador de taxa de aprendizado. cosine indica o uso de um agendador que ajusta a taxa de aprendizado seguindo uma curva cosseno, que decresce gradualmente ao longo do tempo. Isso ajuda a suavizar a convergência para o final do treinamento.

**save_strategy**: Estratégia para salvar os modelos. epoch significa que o modelo será salvo ao final de cada época de treinamento, permitindo a recuperação do treinamento ou avaliação de modelos em diferentes estágios de treinamento.

**logging_steps**: Frequência dos logs de treinamento. Com 10, os logs serão gerados a cada 10 passos de treinamento, ajudando no monitoramento do progresso.

**num_train_epochs**: Número total de épocas de treinamento. 3 indica que o conjunto de dados completo será passado pelo modelo três vezes.

**max_steps**: Número máximo de passos de treinamento. 250 limita o treinamento a no máximo 250 passos, o que pode ser útil para treinamentos experimentais ou para evitar overfitting.

**fp16**: Habilita o uso de precisão mista de ponto flutuante (FP16). True indica que o treinamento será realizado usando FP16, reduzindo o uso de memória e possivelmente aumentando a velocidade de treinamento em hardware compatível.

## Trainer

In [17]:
# Cria o objeto de treinamento do modelo
dsa_trainer = SFTTrainer(model = dsa_model,
                         train_dataset = dsa_dados,
                         peft_config = dsa_peft_config,
                         args = dsa_training_arguments)

Adding EOS to train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (2690 > 2048). Running this sequence through the model will result in indexing errors


Truncating train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Esta linha de código acima está criando uma instância de um SFTTrainer, que é uma classe destinada ao treinamento de modelos de aprendizado de máquina, especificamente modelos de linguagem, utilizando a técnica de Ajuste Fino Supervisionado (Supervised Fine-Tuning - SFT) ou técnicas de adaptação paramétrica como PEFT (Parameter-Efficient Fine-Tuning). Vamos analisar cada um dos argumentos passados para o construtor:

**model = dsa_model**: Especifica o modelo que será treinado. dsa_model é uma variável que contém o modelo de linguagem que você deseja afinar ou adaptar para uma tarefa específica.

**train_dataset = dsa_dados**: Indica o conjunto de dados de treinamento. dsa_dados é uma variável que armazena os dados que serão usados para treinar o modelo. Esses dados devem estar em um formato que o modelo possa entender, geralmente um conjunto de exemplos de texto para tarefas de PLN.

**peft_config = dsa_peft_config**: Fornece configurações específicas para técnicas de Fine-Tuning eficientes em termos de parâmetros, como LoRA ou Mixout. dsa_peft_config pode incluir configurações como o tamanho do rank para adaptações LoRA, entre outros. Isso permite ajustar precisamente como o modelo será adaptado durante o treinamento, focando na eficiência.

**dataset_text_field = "text"** (não é mais necessário): Especifica o campo dentro dos dados de treinamento que contém o texto a ser usado para o treinamento. Indica que cada exemplo no conjunto de dados tem um campo chamado "text" que contém o conteúdo textual relevante para o treinamento.

**args = dsa_training_arguments** : Passa uma série de argumentos de treinamento configurados anteriormente. dsa_training_arguments pode incluir configurações como o tamanho do lote, taxa de aprendizado, estratégia de salvamento de modelo, entre outros. Esses argumentos definem a forma como o treinamento é realizado, incluindo aspectos como otimização e agendamento de taxa de aprendizado.

**tokenizer = dsa_tokenizer** (não é mais necessário): Fornece o tokenizador a ser usado para preparar os dados de treinamento. O dsa_tokenizer é responsável por converter o texto em formatos que o modelo pode processar, como sequências de IDs de tokens.

**packing = False** (não é mais necessário): Define se o empacotamento (packing) de sequências deve ser usado durante o treinamento. O empacotamento pode aumentar a eficiência do treinamento ao agrupar textos de comprimentos semelhantes juntos, mas aqui está definido como False, indicando que essa técnica não será utilizada.

**max_seq_length = 1024** (não é mais necessário): Limita o comprimento máximo de sequências de texto que serão consideradas durante o treinamento. Sequências mais longas que 1024 tokens serão truncadas. Esse parâmetro ajuda a gerenciar o uso de memória e a garantir que todas as entradas sejam de um tamanho gerenciável para o modelo.

In [18]:
%%time
dsa_trainer.train()

Step,Training Loss
10,2.404200
20,2.484800
30,2.255500
40,1.918300
50,1.947800
60,1.989900
70,1.837100
80,1.658900
90,1.845100
100,1.685400


CPU times: user 3min 29s, sys: 269 ms, total: 3min 30s
Wall time: 3min 31s


TrainOutput(global_step=250, training_loss=1.6953210487365722, metrics={'train_runtime': 210.7505, 'train_samples_per_second': 4.745, 'train_steps_per_second': 1.186, 'total_flos': 1793156772556800.0, 'train_loss': 1.6953210487365722})

In [19]:
# Salva o modelo
dsa_trainer.save_model("modelo_ajustado")

## Merge do LLM com o Adaptador LoRa

In [20]:
# Carrega o modelo pré-treinado
modelo = AutoModelForCausalLM.from_pretrained(nome_llm,
                                              torch_dtype = torch.float16,
                                              load_in_8bit = False,
                                              device_map = "auto",
                                              trust_remote_code = True)

Aqui está a explicação de cada parâmetro utilizado na chamada da função acima:

**nome_llm**: Este é o identificador do modelo de linguagem causal pré-treinado que você deseja carregar. O identificador pode ser o nome de um modelo disponível no hub de modelos da Hugging Face ou o caminho para um diretório local onde um modelo treinado está armazenado. Este parâmetro determina qual modelo específico será carregado.

**torch_dtype = torch.float16**: Especifica o tipo de dados de tensor do PyTorch a ser usado ao carregar o modelo. torch.float16 é uma precisão de ponto flutuante de 16 bits, também conhecida como half precision. Usar esse tipo de dados pode reduzir significativamente o uso de memória e aumentar a velocidade de computação em GPUs compatíveis, embora possa resultar em uma pequena perda de precisão.

**load_in_8bit = False**: Este parâmetro determina se o modelo deve ser carregado em uma representação de 8 bits para economizar memória. Definir como False significa que o modelo não será convertido para 8 bits durante o carregamento. Isso é relevante para otimizações de uso de memória, especialmente útil para modelos grandes, mas requer suporte especial do hardware e do software.

**device_map = "auto"**: Indica como o modelo deve ser distribuído entre os dispositivos disponíveis, como GPUs. O valor "auto" permite que a biblioteca automaticamente decida a melhor maneira de distribuir as partes do modelo entre os dispositivos disponíveis para otimizar o uso da memória e a velocidade de computação. Isso é particularmente útil para modelos muito grandes que não cabem em uma única GPU.

**trust_remote_code = True**: Quando definido como True, permite que o código personalizado associado ao modelo pré-treinado (por exemplo, customizações no modelo ou no tokenizador que estão armazenadas remotamente junto com o modelo) seja executado. Isso é útil para carregar modelos que dependem de código personalizado para funcionar corretamente, mas deve ser usado com cautela, pois executar código de fontes remotas não verificadas pode apresentar riscos de segurança.

In [21]:
# Carrega o modelo original e faz o merge com o modelo ajustado
peft_model = PeftModel.from_pretrained(modelo,
                                       "modelo_ajustado",
                                       from_transformers = True,
                                       device_map = "auto")

In [22]:
# Merge
modelo_dsa_final = peft_model.merge_and_unload()

In [23]:
modelo_dsa_final

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), eps=1e-05)
    (rotary_emb): 

## Função Para Obter Resposta do LLM

In [24]:
# Função para formatar o prompt
def dsa_formata_prompt(instruction, input)-> str:
    return f"<|im_start|>user\n{instruction}<|im_end|>\n<|im_start|>input\n{input}<|im_end|>\n<|im_start|>assistant:"

In [25]:
# Define a função que aceita uma instrução e uma entrada do usuário
def dsa_gera_resposta(instruction, user_input):

    # Formata o prompt combinando a instrução e a entrada do usuário
    prompt = dsa_formata_prompt(instruction, user_input)

    # Tokeniza o prompt formatado para preparar a entrada para o modelo
    inputs = dsa_tokenizer([prompt], return_tensors = "pt")

    # Configura os parâmetros para a geração de texto, incluindo penalidades e estratégias de amostragem
    generation_config = GenerationConfig(

        # Define o alpha para a penalidade aplicada a tokens específicos
        penalty_alpha = 0.6,

        # Ativa a amostragem para a geração de texto
        do_sample = True,

        # Limita a geração a partir dos 5 tokens mais prováveis
        top_k = 5,

        # Ajusta a "temperatura" para a diversificação dos resultados
        temperature = 0.4,

        # Aplica penalidade para repetição de tokens
        repetition_penalty = 1.2,

        # Limita o número de novos tokens gerados
        max_new_tokens = 120,

        # Define o ID do token de preenchimento
        pad_token_id = dsa_tokenizer.eos_token_id)

    # Inicia a contagem do tempo para medir a duração da inferência
    start_time = perf_counter()

    # Tokeniza o prompt e move a entrada para a GPU para acelerar a geração
    inputs = dsa_tokenizer(prompt, return_tensors = "pt").to('cuda')

    # Gera a saída usando o modelo e a configuração de geração definida
    outputs = dsa_model.generate(**inputs, generation_config = generation_config)

    # Decodifica a saída do modelo para texto legível, ignorando tokens especiais
    decoded_output = dsa_tokenizer.decode(outputs[0], skip_special_tokens = True).strip()

    # Exibe a saída decodificada
    print(decoded_output)

    # Calcula o tempo total gasto na inferência
    output_time = perf_counter() - start_time

    # Exibe o tempo total de inferência em segundos
    print(f"Tempo Total Para Inferência: {round(output_time, 2)} segundos")

## Medical Natural Language Inference (NLI)

### Inferência 1

In [26]:
instrucao_1 = 'If you have medical expertise, assist the user by responding to their healthcare-related questions.'

In [27]:
input_1 = 'Hi I am 33 year old male , 200 lbs . Sunday I had a sever panic attack , was dizzy , naseuas , almost fainted . Went to the emergency room , had a ekg and blood work all came back normal . I do suffer from anxiety but as of late I have chest pains . Sharp pains in diffrent spots of my chest . They come and go . I also have a strange feeling I m my head and pressure under my rib cage . My blood pressure is always normal , but have a high pulse rate . Ranges from 85 - 111 bpm evertime I check it . Are all of these related to stress , what can be done . A very scared man , Mike'

In [28]:
# Aplica a função
dsa_gera_resposta(instruction = instrucao_1, user_input = input_1)

<|im_start|>user
If you have medical expertise, assist the user by responding to their healthcare-related questions.<|im_end|>
<|im_start|>input
Hi I am 33 year old male , 200 lbs . Sunday I had a sever panic attack , was dizzy , naseuas , almost fainted . Went to the emergency room , had a ekg and blood work all came back normal . I do suffer from anxiety but as of late I have chest pains . Sharp pains in diffrent spots of my chest . They come and go . I also have a strange feeling I m my head and pressure under my rib cage . My blood pressure is always normal , but have a high pulse rate . Ranges from 85 - 111 bpm evertime I check it . Are all of these related to stress , what can be done . A very scared man , Mike<|im_end|>
<|im_start|>assistant: Hi there. It sounds like you've experienced some serious symptoms that are not just due to stress. The first thing we would recommend doing is getting an ECG (electrocardiogram) to rule out any heart issues. If your doctor confirms no cardi

### Inferência 2

In [29]:
instrucao_2 = 'As a medical chatbot, your responsibility is to provide information and guidance on medical matters to users.'

In [30]:
input_2 = 'Can i give my 6 month old 18lb baby anythi g for allergies shes having clear boogars that were green but now are clear again her eyes are watery and she keeps sneezing she cant breathe and drink her bottle and the pedi is not awnsering called 6 times already'

In [31]:
# Aplica a função
dsa_gera_resposta(instruction = instrucao_2, user_input = input_2)

<|im_start|>user
As a medical chatbot, your responsibility is to provide information and guidance on medical matters to users.<|im_end|>
<|im_start|>input
Can i give my 6 month old 18lb baby anythi g for allergies shes having clear boogars that were green but now are clear again her eyes are watery and she keeps sneezing she cant breathe and drink her bottle and the pedi is not awnsering called 6 times already<|im_end|>
<|im_start|>assistant: Hi there. I am Dr. Alicia. Can you please tell me about your daughter's condition? What symptoms does she have? How long has this been going on? Is it just in the past week or so? If yes then it may be an allergy reaction. It would help if you could describe what she looks like when she is sick. Are there any other children around her who might also have similar reactions? Please let me know more details regarding your daughter's health. Thank you.
Tempo Total Para Inferência: 8.05 segundos


### Inferência 3

In [32]:
instrucao_3 =  '''In your capacity as a doctor, it is expected that you answer the medical questions relying on the patient's description.
Analyze the question given its context. Give both long answer and yes/no decision.'''

In [33]:
input_3 = '''###Question: Does systemic inflammation response syndrome score predict the mortality in multiple trauma patients? '''

In [34]:
# Aplica a função
dsa_gera_resposta(instruction = instrucao_3, user_input = input_3)

<|im_start|>user
In your capacity as a doctor, it is expected that you answer the medical questions relying on the patient's description.
Analyze the question given its context. Give both long answer and yes/no decision.<|im_end|>
<|im_start|>input
###Question: Does systemic inflammation response syndrome score predict the mortality in multiple trauma patients? <|im_end|>
<|im_start|>assistant:
###Answer: No
Tempo Total Para Inferência: 0.63 segundos


In [35]:
%watermark -a "Data Science Academy"

Author: Data Science Academy



In [36]:
#%watermark -v -m

In [37]:
#%watermark --iversions

# Fim